In [ ]:
import ipyvuetify as v
import ipywidgets as widgets
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.ticker import FuncFormatter, MultipleLocator
import requests
from IPython.display import display, clear_output
import json

### SQLytes Inventory Tracking

In [ ]:
backend = 'http://127.0.0.1:5000/sqlytes/'

In [ ]:
class WarehouseWidgets:
    def __init__(self):
        self.output_area = widgets.Output()
        self.available_warehouses = self.getAvailableWarehouses()
        self.warehouses = self.getAllWarehouses().get('Warehouses')
        self.current_warehouse_index = 0
        self.all_warehouses_df = pd.DataFrame()
       
        self.warehouses_options_dropdown = widgets.Select(
            options=['All Warehouses info', 'All parts for a warehouse'],
            value='All Warehouses info',
            description='Search for:',
            rows=2,
        )

        self.warehouse_selection = widgets.Dropdown(
            options=self.available_warehouses,
            value=self.available_warehouses[0],
            description="Select wid:"
        )


        self.next_warehouse_button = widgets.Button(
            description="Next Warehouse",
            icon='check'
        )
        
    def getAvailableWarehouses(self):
        """Return all IDs of the warehouses in the Data Base."""
        response = requests.get(f'{backend}{"warehouse"}')
        # List to store all available wid's.
        wids = []
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            for record in data['Warehouses']:
                wids.append(record['wid'])
        else:
            print('Error {}: Could not get data from the backend.'.format(response.status_code))
        return wids
    
    def getAllWarehouses(self):
        """Return all warehouses in the Data Base."""
        response = requests.get(f'{backend}{"warehouse"}')
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            return data

        print(f'Error {response.status_code}: Could not get data from the backend.')
        return {}
    
    def displayNextWarehouse(self,_):
        self.current_warehouse_index = (self.current_warehouse_index+1) % len(self.warehouses)
        print(f"Displaying index {self.current_warehouse_index}")
        selected_warehouse = self.warehouses[self.current_warehouse_index]
        print(f"Displaying Warehouse ID: {selected_warehouse}")
        selected_warehouse_df = pd.DataFrame.from_dict(selected_warehouse, orient="index")
        self.all_warehouses_df = selected_warehouse_df
        with self.output_area:
            clear_output(wait=True)
            display(self.all_warehouses_df)
            
    def getWarehousePartsAPI(self, selected_warehouse):
        """Obtain all Parts for a warehouse.

        Args:
            wid (int): ID of the warehouse of which the parts are going to be showed.
            endpoint (str): Endpoint to be connected to the backend route to obtain the data.
        """
        endpoint = f'warehouse/{selected_warehouse}/parts'
        response = requests.get(f'{backend}{endpoint}')
        with self.output_area:
            clear_output(wait=True)  # clear cell
            if response.status_code == 200: 
                data = response.json()
                all_parts_df = pd.DataFrame(data["WarehouseParts"])
                display(all_parts_df)
            elif response.status_code == 404:
                print('Error {}: Warehouse has no parts'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))

    def on_selection_change(self, change):
        if isinstance(change, int):
            # Directly received the selected value, not an event dictionary
            selected_warehouse = change
        elif change['type'] == 'change' and change['name'] == 'value':
            # Received an event dictionary, extract the selected value
            selected_warehouse = change['new']
        else:
            # Invalid change, do nothing
            return
        self.getWarehousePartsAPI(selected_warehouse)


### Select Warehouse to search the parts for

In [ ]:
warehouseWidgets = WarehouseWidgets()

In [ ]:
warehouseWidgets.getWarehousePartsAPI(warehouseWidgets.warehouse_selection.value)
display(warehouseWidgets.warehouse_selection)
display(warehouseWidgets.output_area)

In [ ]:
class SuppliedPartsWidgets:
    def __init__(self):
        self.output_area = widgets.Output()
        self.available_suppliers = self.getAvailableSuppliers()
        self.suppliers = self.getAllSuppliers()
        self.current_supplier_index = 0
        self.all_suppliers_df = pd.DataFrame()
       
        self.suppliers_options_dropdown = widgets.Select(
            options=['All Suppliers info', 'All parts supplied by this supplier'],
            value='All Suppliers info',
            description='Search for:',
            rows=2,
        )

        self.supplier_selection = widgets.Dropdown(
            options=self.available_suppliers,
            value=self.available_suppliers[0],
            description="Select sid:"
        )
        
        self.supplier_selection.observe(self.on_selection_change)
        
        self.next_supplier_button = widgets.Button(
            description="Next Supplier",
            icon='check'
        )
        
    def getAvailableSuppliers(self):
        """Return all IDs of the suppliers in the Data Base."""
        response = requests.get(f'{backend}{"supplier"}')
        # List to store all available sid's.
        sids = []
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            for record in data:
                sids.append(record['id'])
        else:
            print('Error {}: Could not get data from the backend.'.format(response.status_code))
        return sids
    
    def getAllSuppliers(self):
        """Return all suppliers in the Data Base."""
        response = requests.get(f'{backend}{"supplier"}')
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            return data

        print(f'Error {response.status_code}: Could not get data from the backend.')
        return {}
    
    def displayNextSupplier(self,_):
        self.current_supplier_index = (self.current_supplier_index+1) % len(self.suppliers)
        print(f"Displaying index {self.current_supplier_index}")
        selected_supplier = self.suppliers[self.current_supplier_index]
        print(f"Displaying Warehouse ID: {selected_supplier}")
        selected_supplier_df = pd.DataFrame.from_dict(selected_supplier, orient="index")
        self.all_suppliers_df = selected_supplier_df
        with self.output_area:
            clear_output(wait=True)
            display(self.all_suppliers_df)
            
    def getSuppliedPartsAPI(self,selected_supplier):
        """Obtain all Parts for a warehouse.

        Args:
            wid (int): ID of the warehouse of which the parts are going to be showed.
            endpoint (str): Endpoint to be connected to the backend route to obtain the data.
        """
        endpoint = f'supplies/{selected_supplier}'
        response = requests.get(f'{backend}{endpoint}')
        with self.output_area:
            clear_output(wait=True)  # Clear the previous output
            if response.status_code == 200: 
                data = response.json()
                supplied_parts_df = pd.DataFrame(data)
                display(supplied_parts_df)
            elif response.status_code == 404:
                print('Error {}: Supplier does not supply any parts'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))
                
    def on_selection_change(self,change):
        if isinstance(change, int):
            # Directly received the selected value, not an event dictionary
            selected_supplier = change
        elif change['type'] == 'change' and change['name'] == 'value':
            # Received an event dictionary, extract the selected value
            selected_supplier = change['new']
        else:
            # Invalid change, do nothing
            return
        self.getSuppliedPartsAPI(selected_supplier)

### Select supplier to search the parts it supplies

In [ ]:
suppliedPartsWidgets = SuppliedPartsWidgets()

In [ ]:
suppliedPartsWidgets.getSuppliedPartsAPI(suppliedPartsWidgets.supplier_selection.value)
suppliedPartsWidgets.supplier_selection.observe(suppliedPartsWidgets.on_selection_change)
display(suppliedPartsWidgets.supplier_selection)
display(suppliedPartsWidgets.output_area)

### Select wid to show the transactions for that warehouse

In [ ]:
class TransactionWidgets:
    def __init__(self):
        self.output_area = widgets.Output()
        self.available_warehouses = self.getAvailableWarehouses()
        self.warehouses = self.getAllWarehouses()
        self.current_warehouse_index = 0
        self.all_transactions_df = pd.DataFrame()
       
        self.warehouse_options_dropdown = widgets.Select(
            options=['All Warehouses info', 'All transactions in this warehouse'],
            value='All Warehouses info',
            description='Search for:',
            rows=2,
        )

        self.warehouse_selection = widgets.Dropdown(
            options=self.available_warehouses,
            value=self.available_warehouses[0],
            description="Select wid:"
        )
        
        self.warehouse_selection.observe(self.on_selection_change)
        
        self.next_warehouse_button = widgets.Button(
            description="Next Warehouse",
            icon='check'
        )
        

    def getAvailableWarehouses(self):
        """
        Return all IDs of the warehouses in the DB
        """
        response = requests.get(f'{backend}{"warehouse"}')
        # List to store all available sid's.
        wids = []
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            for record in data['Warehouses']:
                wids.append(record['wid'])
        else:
            print('Error {}: Could not get data from the backend.'.format(response.status_code))
        return sorted(wids)

    
    def getAllWarehouses(self):
        """
        Return all suppliers in the Data Base.
        """
        response = requests.get(f'{backend}{"warehouse"}')
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            return data
        print(f'Error {response.status_code}: Could not get data from the backend.')
        return {}
                
    def getTransactionsPerWarehouseAPI(self, selected_warehouse):
        endpoint = f'transaction'
        response = requests.put(f'{backend}{endpoint}', data=json.dumps({"wid": selected_warehouse}), headers={"Content-Type": "application/json"})
        with self.output_area:
            clear_output(wait=True)  # Clear the previous output
            if response.status_code == 200: 
                data = response.json()
                supplied_parts_df = pd.DataFrame(data["Result"])
                display(supplied_parts_df)
            elif response.status_code == 404:
                print('Error {}: Warehouse does not have any transactions'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))
                
    def on_selection_change(self,change):
        if isinstance(change, int):
            # Directly received the selected value, not an event dictionary
            selected_warehouse = change
        elif change['type'] == 'change' and change['name'] == 'value':
            # Received an event dictionary, extract the selected value
            selected_warehouse    = change['new']
        else:
            # Invalid change, do nothing
            return
        self.getTransactionsPerWarehouseAPI(selected_warehouse)


In [ ]:
transactionWidget = TransactionWidgets()
transactionWidget.getTransactionsPerWarehouseAPI(transactionWidget.warehouse_selection.value)
transactionWidget.warehouse_selection.observe(transactionWidget.on_selection_change)
display(transactionWidget.warehouse_selection)
display(transactionWidget.output_area)

# Local Statistics

In [ ]:
# For graphs
def currency_formatter(float, pos):
    return f"${float:.2f}"

In [ ]:
class LocalStatsWidgets:
    def __init__(self):
        self.yearly_output = widgets.Output()
        self.capacity_output = widgets.Output()
        self.material_output = widgets.Output()
        self.expensive_output = widgets.Output()
        self.suppliers_output = widgets.Output()
        self.days_output = widgets.Output()
        self.receives_output = widgets.Output()
        self.available_warehouses = self.getAvailableWarehouses()
        self.outputs = {self.yearly_output: self.getYearlyProfit,
                        self.capacity_output: self.getCapacityUnderThresh,
                        self.material_output: self.getBottomMaterials,
                        self.expensive_output: self.getExpensiveRacks,
                        self.suppliers_output: self.getTopSuppliers,
                        self.days_output: self.getSmallestIncomingCost,
                        self.receives_output: self.getMostExchanges}

        self.wid_input = widgets.Text(value="", placeholder="Enter warehouse ID", description="Enter wid:", disabled=False)
        self.uid_input = widgets.Text(value="", placeholder="Enter User ID", description="Enter uid:", disabled=False)
        self.submit_button = widgets.Button(description="Run query")

    def getAvailableWarehouses(self):
        """Return all IDs of the warehouses in the Data Base."""
        response = requests.get(f'{backend}{"warehouse"}')
        # List to store all available wid's.
        wids = []
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            for record in data['Warehouses']:
                wids.append(record['wid'])
        else:
            print('Error {}: Could not get data from the backend.'.format(response.status_code))
        return wids


    def user_works(self, wid, uid):
        response = requests.get(f'{backend}{"warehouse/"}{uid}/worksin/{wid}')
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            return data
        print(f'Error {response.status_code}: Could not get data from the backend.')
        return {}

    def validateLocalStatsConnection(self, response, wid, uid, url):
        if response.status_code == 404:
            if wid not in self.available_warehouses:
                display(f"Warehouse {wid} not valid.")
            else:
                can_access = self.user_works(wid, uid)
                if not can_access:
                    display(f'Error {response.status_code}: User {uid} cannot work in Warehouse #{wid}.')
                else:
                    display(f'Error {response.status_code}: Warehouse #{wid} has no relevant data.')
        else:
            display(f'{url} accessed with user #{uid}.')
            display(f'Error {response.status_code}: Could not get data from the backend.')

    def getYearlyProfit(self):
        selected_warehouse, selected_user = int(self.wid_input.value), int(self.uid_input.value)
        endpoint = f'warehouse/{selected_warehouse}/profit'
        url = f'{backend}{endpoint}'
        response = requests.post(url, json={"user": selected_user})
        with self.yearly_output:
            clear_output(wait=True)
            if response.status_code == 200:
                data = response.json()
                # Plot
                sorted_data = sorted(data['Yearly Profit'], key=lambda x: x['Year'])
                years = [entry['Year'] for entry in sorted_data]
                net_profits = [entry['Net Profit'] for entry in sorted_data]
                plt.plot(years, net_profits, marker='o', linestyle='-')
                plt.xlabel('Year')
                plt.ylabel('Net Profit')
                plt.title(f'Yearly Profit for Warehouse #{selected_warehouse}')
                # Formats money
                plt.gca().yaxis.set_major_formatter(FuncFormatter(currency_formatter))
                plt.show()
            else:
                self.validateLocalStatsConnection(response, selected_warehouse, selected_user, url)

    def getCapacityUnderThresh(self):
        selected_warehouse, selected_user = int(self.wid_input.value), int(self.uid_input.value)
        endpoint = f'warehouse/{selected_warehouse}/rack/lowstock'
        url = f'{backend}{endpoint}'
        response = requests.post(url, json={"user": selected_user})
        with self.capacity_output:
            clear_output(wait=True)
            if response.status_code == 200:
                data = response.json()
                # Plot
                racks = [entry['Rack'] for entry in data['Lowest Threshold Racks']]
                thresholds = [float(entry['Low Capacity Threshold']) for entry in data['Lowest Threshold Racks']]
                parts_quantity = [entry['Parts Quantity'] for entry in data['Lowest Threshold Racks']]
                fig, ax = plt.subplots()
                bars = ax.bar(racks, parts_quantity)
                ax.set_xlabel('Rack Names')
                ax.set_ylabel('Threshold')
                ax.set_xticks(range(len(racks)))
                ax.set_xticklabels(racks, rotation=45, ha='right')
                for bar, part_quantity in zip(bars, parts_quantity):
                    height = bar.get_height()
                    ax.text(bar.get_x() + bar.get_width()/2, height, str(part_quantity), ha='center', va='bottom')
                max_threshold = max(thresholds)
                ax.set_ylim(0, max_threshold)
                num_major_ticks = 5
                ax.yaxis.set_major_locator(MultipleLocator(max_threshold/(num_major_ticks-1)))
                plt.title('Top 5 Racks Under the 25% Capacity Threshold')
                plt.show()
            else:
                self.validateLocalStatsConnection(response, selected_warehouse, selected_user, url)

    def getBottomMaterials(self):
        selected_warehouse, selected_user = int(self.wid_input.value), int(self.uid_input.value)
        endpoint = f'warehouse/{selected_warehouse}/rack/material'
        url = f'{backend}{endpoint}'
        response = requests.post(url, json={"user": selected_user})
        with self.material_output:
            clear_output(wait=True)
            if response.status_code == 200:
                data = response.json()
                # Plot
                parts_data = data['Bottom Racks']
                rack_names = [f"{part['Part']}" for part in parts_data]
                part_counts = [part['Part Count'] for part in parts_data]
                types = list(set(part['Type'] for part in parts_data))
                # Creating a color map for each rack type
                colors = [cm.tab10(i) for i in range(len(types))]
                fig, ax = plt.subplots()
                for i, type_name in enumerate(types):
                    indices = [j for j, part in enumerate(parts_data) if part['Type'] == type_name]
                    ax.bar([rack_names[j] for j in indices], [part_counts[j] for j in indices], color=colors[i], label=type_name)
                ax.set_xlabel('Rack Names')
                ax.set_ylabel('Part Count')
                plt.title('Bottom 3 Part Counts per Type, in Racks')
                plt.xticks(rotation=45, ha='right')
                plt.legend(title='Type', bbox_to_anchor=(1, 1), loc='upper left')
                plt.show()
            else:
                self.validateLocalStatsConnection(response, selected_warehouse, selected_user, url)

    def getExpensiveRacks(self):
        selected_warehouse, selected_user = int(self.wid_input.value), int(self.uid_input.value)
        endpoint = f'warehouse/{selected_warehouse}/rack/expensive'
        url = f'{backend}{endpoint}'
        response = requests.post(url, json={"user": selected_user})
        with self.expensive_output:
            clear_output(wait=True)
            if response.status_code == 200:
                data = response.json()
                # Plot
                racks_data = data['Most Expensive Racks']
                rack_names = [rack['Rack'] for rack in racks_data]
                rack_prices = [rack['Rack Price'] for rack in racks_data]
                fig, ax = plt.subplots()
                bars = ax.bar(rack_names, rack_prices)
                for bar, rack_price in zip(bars, rack_prices):
                    height = bar.get_height()
                    ax.text(bar.get_x() + bar.get_width()/2, height, f'${rack_price:.2f}', ha='center', va='bottom')
                ax.set_xlabel('Rack Names')
                ax.set_ylabel('Rack Prices')
                plt.title('Top 5 Most Expensive Racks')
                plt.gca().yaxis.set_major_formatter(FuncFormatter(currency_formatter))
                plt.xticks(rotation=45, ha='right')
                plt.show()
            else:
                self.validateLocalStatsConnection(response, selected_warehouse, selected_user, url)

    def getTopSuppliers(self):
        selected_warehouse, selected_user = int(self.wid_input.value), int(self.uid_input.value)
        endpoint = f'warehouse/{selected_warehouse}/transaction/suppliers'
        url = f'{backend}{endpoint}'
        response = requests.post(url, json={"user": selected_user})
        with self.suppliers_output:
            clear_output(wait=True)
            if response.status_code == 200:
                data = response.json()
                # Plot
                suppliers_data = data['Top Warehouse Suppliers']
                supplier_names = [supplier['Supplier Name'] for supplier in suppliers_data]
                supply_counts = [supplier['Supply Count'] for supplier in suppliers_data]
                fig, ax = plt.subplots()
                ax.bar(supplier_names, supply_counts)
                ax.set_xlabel('Supplier Names')
                ax.set_ylabel('Supply Count')
                plt.title(f'Top 3 Suppliers for Warehouse #{selected_warehouse}')
                plt.xticks(rotation=45, ha='right')
                plt.show()
            else:
                self.validateLocalStatsConnection(response, selected_warehouse, selected_user, url)

    def getSmallestIncomingCost(self):
        selected_warehouse, selected_user = int(self.wid_input.value), int(self.uid_input.value)
        endpoint = f'warehouse/{selected_warehouse}/transaction/leastcost'
        url = f'{backend}{endpoint}'
        response = requests.post(url, json={"user": selected_user})
        with self.days_output:
            clear_output(wait=True)
            if response.status_code == 200:
                data = response.json()
                # Plot
                costs_data = data['Least Incoming Trans. Costs']
                dates = [datetime.strptime(entry['Transaction Date'], '%a, %d %b %Y %H:%M:%S %Z') for entry in costs_data]
                total_costs = [entry['Total Incoming Cost'] for entry in costs_data]
                fig, ax = plt.subplots()
                ax.plot(dates, total_costs, marker='o', linestyle='-', color='b')
                ax.set_xlabel('Transaction Dates')
                ax.set_ylabel('Total Incoming Costs')
                ax.tick_params(axis='x', rotation=30)
                # Setting x-axis, do something when there isn't enough data
                date_range = max(dates) - min(dates)
                if date_range.days > 1:
                    ax.set_xlim(min(dates), max(dates))
                else:
                    ax.set_xlim(min(dates) - pd.DateOffset(days=1), max(dates) + pd.DateOffset(days=1))
                plt.title('Top 3 Days with Smallest Incoming Transaction Costs over Time')
                plt.show()
            else:
                self.validateLocalStatsConnection(response, selected_warehouse, selected_user, url)

    def getMostExchanges(self):
        selected_warehouse, selected_user = int(self.wid_input.value), int(self.uid_input.value)
        endpoint = f'warehouse/{selected_warehouse}/users/receivesmost'
        url = f'{backend}{endpoint}'
        response = requests.post(url, json={"user": selected_user})
        with self.receives_output:
            clear_output(wait=True)
            if response.status_code == 200:
                data = response.json()
                # Plot
                user_data = data['Most User Exchanges']
                user_names = [f"{user['First Name']} {user['Last Name']}" for user in user_data]
                transfer_counts = [user['Transfer Count'] for user in user_data]
                fig, ax = plt.subplots()
                ax.bar(user_names, transfer_counts)
                ax.set_xlabel('Users')
                ax.set_ylabel('Transfer Count')
                plt.title('Top 3 Users that Receive the Most Exchanges')
                plt.xticks(rotation=45, ha='right')
                plt.show()
            else:
                self.validateLocalStatsConnection(response, selected_warehouse, selected_user, url)

In [ ]:
localStatsWidget = LocalStatsWidgets()

In [ ]:
def display_all_stats(b):
    for output in localStatsWidget.outputs:
        execute = localStatsWidget.outputs.get(output)
        with output:
            execute()

### Run All Local Statistics

In [ ]:
display(localStatsWidget.wid_input)
display(localStatsWidget.uid_input)
display(localStatsWidget.submit_button)
# Runs all statistics at once
localStatsWidget.submit_button.on_click(display_all_stats)
for out in localStatsWidget.outputs:
    display(out)